# Models

> Collection of Deep Learning models

In [ ]:
#| default_exp models

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2
from nbdev.showdoc import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| export
import torch.nn.functional as F
import torch.nn as nn
import torch
from nimrod.modules import Encoder, Decoder
from pytorch_lightning import LightningModule

In [ ]:
#| export
class AutoEncoder(nn.Module):
    def __init__(self,
        encoder:Encoder, # Encoder layer
        decoder:Decoder # Decoder layer
        ):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
    
    def forward(self,
        x:torch.Tensor # Tensor B x L
        )->torch.Tensor:
        z = self.encoder(x)
        x_hat = self.decoder(z)
        return x_hat

In [ ]:
enc = Encoder()
dec = Decoder()
a = AutoEncoder(enc, dec)
batch = torch.rand((10, 28*28))
y = a(batch)
print(y.shape)

torch.Size([10, 784])


In [ ]:
from nimrod.data.datasets import MNISTDataset
from torch.utils.data import DataLoader

ds = MNISTDataset()
dl = DataLoader(ds)
b = next(iter(dl))
print(len(b), b[0].shape, b[1].shape)


2 torch.Size([1, 1, 28, 28]) torch.Size([1])


In [ ]:
#| export
class AutoEncoderPL(LightningModule):
    def __init__(self, autoencoder:AutoEncoder):
        super().__init__()
        # self.save_hyperparameters()
        self.autoencoder = autoencoder
        self.metric = torch.nn.MSELoss()

    def forward(self, x):
        return self.autoencoder(x)

    def _shared_eval(self, batch, batch_idx, prefix, prog_bar=True, on_step=False, on_epoch=True, sync_dist=True):
        x, _ = batch
        x = x.view(x.size(0), -1) # flatten B x C x H x W to B x L (grey pic)
        x_hat = self.autoencoder(x)
        loss = self.metric(x_hat, x)
        self.log(f"{prefix}_loss", loss, on_step=on_step, on_epoch=on_epoch, sync_dist=sync_dist)
        return loss

    def training_step(self, batch, batch_idx):
        return self._shared_eval(batch, batch_idx, "train", sync_dist=False)
    
    def test_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "test")
    
    def validation_step(self, batch, batch_idx):
        self._shared_eval(batch, batch_idx, "val")

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer



In [ ]:
autoencoder_pl = AutoEncoderPL(a)
b = torch.rand((5,28*28))
y = autoencoder_pl(b)
print(y.shape)

torch.Size([5, 784])


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()